In [239]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

In [244]:
# Beregn kaste længden af det stå kast
TYNGEACCELERATION = 9.82  # m/s^2
START_HASTIGHED = 20     # m/s
START_HØJDE = 0.5         # m
TIME_STEP_SIZE = 0.001    # s

def tyngdekraft(vektor:int):
    N = np.shape(vektor)[1]
    gx = np.zeros([1,N])
    gy = -TYNGEACCELERATION * np.ones([1,N])
    
    return np.vstack((gx,gy))

def definer_modstandskraft(masse, cv, densitet, A):
    modstands_koefficient = 0.5 * cv * densitet * A / masse
    
    # Vindmodstand
    fart = lambda hastighed: np.sqrt(hastighed[0,:]**2 + hastighed[1,:]**2)
    vindmodstand = lambda hastighed: (-modstands_koefficient * fart(hastighed)) * hastighed
   
    # Definer modstandskraft
    modstands_acceleration = lambda hastighed: tyngdekraft(hastighed) + vindmodstand(hastighed)
    return modstands_acceleration

def setup_beregninger():
    """Importer parametre og setup beregnere"""
    masse        = parametres['masse']
    start_vinkel = parametres['initial angle']
    cv           = parametres['form faktor']
    A            = parametres['tværsnits areal']
    densitet     = parametres['væske densitet']
    
    # Definer vigtige objekter
    acceleration  = definer_modstandskraft(masse, cv, densitet, A)
    
    angle_in_radians = np.pi / 180 * start_vinkel
    vx = START_HASTIGHED * np.cos(angle_in_radians)
    vy = START_HASTIGHED * np.sin(angle_in_radians)
    start_hastighed = np.vstack((vx, vy))
    
    return start_hastighed, acceleration

def kør_algoritme(start_hastighed, acceleration):
    """Beregn kaste kurven for det skrå kast"""
    N = 10000
    
    #Pre-allocate
    kaste_tid = [0 for _ in range(N)]
    x = [0 for _ in range(N)]
    y = [0 for _ in range(N)]
    
    # Initial conditions
    n = 0
    kaste_tid[0] = 0
    x[0] = 0
    y[0] = START_HØJDE
    hastighed = start_hastighed
    
    # Simulation
    allocated = True
    while y[n] > 0:
        # Første koefficienter
        k1 = TIME_STEP_SIZE * acceleration(hastighed)
        j1 = TIME_STEP_SIZE * hastighed
        
        # Anden koefficienter
        k2 = TIME_STEP_SIZE * acceleration(hastighed + 0.5*j1)
        j2 = TIME_STEP_SIZE * (hastighed + 0.5*k1)
        
        # Tredje koefficienter
        k3 = TIME_STEP_SIZE * acceleration(hastighed + 0.5*j2)
        j3 = TIME_STEP_SIZE * (hastighed + 0.5*k2)
        
        # Fjerde koefficienter
        k4 = TIME_STEP_SIZE * acceleration(hastighed + j3)
        j4 = TIME_STEP_SIZE * (hastighed + k3)
        
        # Step
        hastighed += (k1 + 2*(k2 + k3) + k4) / 6
        positions_step = (j1 + 2*(j2 + j3) + j4) / 6
        
        if allocated:
            try:
                kaste_tid[n+1] = kaste_tid[n] + TIME_STEP_SIZE
                x[n+1] = x[n] + positions_step[0]
                y[n+1] = y[n] + positions_step[1]
            except IndexError:
                kaste_tid.append(kaste_tid[-1] + TIME_STEP_SIZE)
                x.append(float(x[n] + positions_step[0]))
                y.append(float(y[n] + positions_step[1]))
                allocated = False
        else:
            kaste_tid.append(kaste_tid[-1] + TIME_STEP_SIZE)
            x.append(float(x[-1] + positions_step[0]))
            y.append(float(y[-1] + positions_step[1]))
        n += 1
    
    kaste_tid = kaste_tid[0:n+1]
    x = x[0:n+1]
    y = y[0:n+1]
    
    return kaste_tid, x, y

def visualizer_resultater(kaste_tid, x, y, grid):
    """Lav en graf der viser kastelængden"""
    
    fig, ax = plt.subplots()
    ax.plot(x, y, color='blue')
    plt.xlabel('Afstand i x')
    plt.ylabel('Afstand i y')
    plt.xlim([0, 45])
    plt.grid(grid)
    
    
    #Estimer slutpunkt
    x1 = float(x[-2])
    x2 = float(x[-1]) 
    y1 = float(y[-2]) 
    y2 = float(y[-1])
    
    a = (y2 - y1) / (x2 - x1)
    b = y2 - a*x2
    
    kaste_længde = -b/a
    
    print(f'Kaste tid = {kaste_tid[-1]} s')
    print(f'Kaste længe = {kaste_længde} m')
    

def kør_simulering(grid):
    """Beregn kastekurven for det skrå kast med vindmodstand"""
    start_hastighed, acceleration = setup_beregninger()
    kaste_tid, x, y = kør_algoritme(start_hastighed, acceleration)
    visualizer_resultater(kaste_tid, x, y, grid)

In [241]:
def materiale_egenskaber(bredte, form, materiale):
    rumfang = form_egenskaber(form, bredte)
    densitet = materiale_densitet(materiale)
    parametres['masse'] = densitet * rumfang
    
def form_egenskaber(form, bredte) -> float:
    form = form.lower()
    if form == 'kugle':
        cv = 0.47
        A = np.pi * bredte**2
        V = 4/3 * np.pi * bredte**3
    elif form == 'halv kugle':
        cv = 0.42
        A = np.pi * bredte**2
        V = 2/3 * np.pi * bredte**3
    elif form == 'kegle':
        cv = 0.50
        A = np.pi * bredte**2
        V = 1/3 * np.pi * bredte**3
        
    parametres['form faktor'] = cv
    parametres['tværsnits areal'] = A
    return V

def materiale_densitet(materiale: str) -> float:
    materiale = materiale.lower()
    
    # Materialer til væske
    if materiale == 'vand':
        return 1000
    if materiale == 'honning':
        return 1400
    if materiale == 'luft':
        return 1.225
    
    # Materialer til projektil
    if materiale == 'candyfloss':
        return 18
    if materiale == 'guld':
        return 19300
    if materiale == 'bly':
        return 11340
    if materiale == 'birk':
        return 650
    if materiale == 'bøg':
        return 680
    if materiale == 'ahorn':
        return 560

In [248]:
parametres = {}

@widgets.interact_manual(
    form=['kugle', 'halv kugle', 'kegle'],
    materiale=['candyfloss', 'guld', 'bly', 'birk', 'bøg', 'ahorn'],
    væske = ['vand', 'honning', 'luft'],
    form_bredte=(10, 30),
    start_vinkel=(0,90)
)
def kør(form='kugle', materiale='stål', væske='luft', form_bredte=10, start_vinkel=30, grid=True):
    form_bredte = 0.01 * form_bredte
    
    # Få egenskaberne for materialet
    materiale_egenskaber(form_bredte, form, materiale)
    parametres['initial angle'] = start_vinkel
    parametres['væske densitet'] = materiale_densitet(væske)
    
    kør_simulering(grid)

interactive(children=(Dropdown(description='form', options=('kugle', 'halv kugle', 'kegle'), value='kugle'), D…